In [31]:
import time
import multiprocessing
import pandas as pd
import nltk
from gensim.models.phrases import Phrases, Phraser
from nltk.tokenize import TreebankWordTokenizer, TweetTokenizer
nltk.download('punkt')
sent_detector = nltk.data.load('tokenizers/punkt/english.pickle')
from tqdm import tqdm
from gensim.models.phrases import Phrases, Phraser
from gensim.models import Word2Vec
from multiprocessing import cpu_count

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\amale\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


### Pré process des données 

In [13]:
df_pol = pd.read_csv('1- données/Political-media-DFE.csv', encoding='latin-1')
df_eco = pd.read_csv('1- données/Full-Economic-News-DFE-839861.csv', encoding='latin-1')
df_movies = pd.read_csv('1- données/movies_metadata.csv')

C:\Users\amale\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3071: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [16]:
df_pol.text.iloc[0]

'RT @nowthisnews: Rep. Trey Radel (R- #FL) slams #Obamacare. #politics https://t.co/zvywMG8yIH'

In [17]:
df_eco.text.iloc[0]

'NEW YORK -- Yields on most certificates of deposit offered by major banks dropped more than a tenth of a percentage point in the latest week, reflecting the overall decline in short-term interest rates.</br></br>On small-denomination, or "consumer," CDs sold directly by banks, the average yield on six-month deposits fell to 5.49% from 5.62% in the week ended yesterday, according to an 18-bank survey by Banxquote Money Markets, a Wilmington, Del., information service.</br></br>On three-month "consumer" deposits, the average yield sank to 5.29% from 5.42% the week before, according to Banxquote. Two banks in the Banxquote survey, Citibank in New York and CoreStates in Pennsylvania, are paying less than 5% on threemonth small-denomination CDs.</br></br>Declines were somewhat smaller on five-year consumer CDs, which eased to 7.37% from 7.45%, Banxquote said.</br></br>Yields on three-month and six-month Treasury bills sold at Monday\'s auction plummeted more than a fifth of a percentage po

In [18]:
df_movies.overview.iloc[0]

"Led by Woody, Andy's toys live happily in his room until Andy's birthday brings Buzz Lightyear onto the scene. Afraid of losing his place in Andy's heart, Woody plots against Buzz. But when circumstances separate Buzz and Woody from their owner, the duo eventually learns to put aside their differences."

In [20]:
def remove_hashtags(tokens):
  tokens = map(lambda x: x.replace('#', ''), tokens)
  return list(tokens)

def remove_url(tokens):
  tokens = filter(lambda x: "http" not in x, tokens)
  return list(tokens)

def remove_html(tokens):
  tokens = filter(lambda x: x[0]+x[-1] != '<>', tokens)
  return list(tokens)

In [21]:
remove_hashtags(TweetTokenizer().tokenize(df_pol.text.iloc[0]))

['RT',
 '@nowthisnews',
 ':',
 'Rep',
 '.',
 'Trey',
 'Radel',
 '(',
 'R',
 '-',
 'FL',
 ')',
 'slams',
 'Obamacare',
 '.',
 'politics',
 'https://t.co/zvywMG8yIH']

In [22]:
remove_url(TweetTokenizer().tokenize(df_pol.text.iloc[0]))

['RT',
 '@nowthisnews',
 ':',
 'Rep',
 '.',
 'Trey',
 'Radel',
 '(',
 'R',
 '-',
 '#FL',
 ')',
 'slams',
 '#Obamacare',
 '.',
 '#politics']

In [23]:
remove_html(TweetTokenizer().tokenize(df_eco.text.iloc[0]))

['NEW',
 'YORK',
 '-',
 '-',
 'Yields',
 'on',
 'most',
 'certificates',
 'of',
 'deposit',
 'offered',
 'by',
 'major',
 'banks',
 'dropped',
 'more',
 'than',
 'a',
 'tenth',
 'of',
 'a',
 'percentage',
 'point',
 'in',
 'the',
 'latest',
 'week',
 ',',
 'reflecting',
 'the',
 'overall',
 'decline',
 'in',
 'short-term',
 'interest',
 'rates',
 '.',
 'On',
 'small-denomination',
 ',',
 'or',
 '"',
 'consumer',
 ',',
 '"',
 'CDs',
 'sold',
 'directly',
 'by',
 'banks',
 ',',
 'the',
 'average',
 'yield',
 'on',
 'six-month',
 'deposits',
 'fell',
 'to',
 '5.49',
 '%',
 'from',
 '5.62',
 '%',
 'in',
 'the',
 'week',
 'ended',
 'yesterday',
 ',',
 'according',
 'to',
 'an',
 '18',
 '-',
 'bank',
 'survey',
 'by',
 'Banxquote',
 'Money',
 'Markets',
 ',',
 'a',
 'Wilmington',
 ',',
 'Del',
 '.',
 ',',
 'information',
 'service',
 '.',
 'On',
 'three-month',
 '"',
 'consumer',
 '"',
 'deposits',
 ',',
 'the',
 'average',
 'yield',
 'sank',
 'to',
 '5.29',
 '%',
 'from',
 '5.42',
 '%',
 't

In [25]:
def tokenize_url_hashtags(corpus, tweets=False):
  if tweets:
    tokenizer = TweetTokenizer()
  else:
    tokenizer = TreebankWordTokenizer()  
    # Life hack : treebank word tokenizer won't keep html code in one token.
    # To preprocess economics news corpus, use tweettokenizer. 
  tokenized_sentences = []
  for sample in tqdm(corpus):
    # separating sentences
    for sentence in sent_detector.tokenize(sample):
      tokens = tokenizer.tokenize(sentence)
      tokens = remove_url(tokens)
      tokens = remove_html(tokens)
      tokens = remove_hashtags(tokens)
      tokens = list(map(lambda x: x.lower(), tokens))
      tokenized_sentences.append(tokens)
  return tokenized_sentences

In [26]:
cleaned_pol = tokenize_url_hashtags(df_pol.text.array, tweets=True)
cleaned_eco = tokenize_url_hashtags(df_eco.text.array, tweets=False)
cleaned_movie = tokenize_url_hashtags(df_movies.overview.dropna().array)

100%|██████████| 44512/44512 [00:38<00:00, 1159.49it/s]


In [27]:
# we suggest using the Phrases module to train the multi-word detector then iterating on the output to display the results (first 15 for instance)
def show_phrases(tokenized_sentences, threshold=10, shown=15):
  # Training the multi-word expression detector
  phrases = Phrases(tokenized_sentences, threshold=threshold)
  i = 0
  for phrase, score in phrases.export_phrases(tokenized_sentences):
    if i>shown:
      break
    else:
      print("Expression : {0}, score = {1}".format(phrase, score))
    i=i+1

In [28]:
def clean_corpus(corpus, threshold=50, tweets=False):
  tokenized_sentences = tokenize_url_hashtags(corpus, tweets=tweets)
  phrases = Phrases(tokenized_sentences, threshold=threshold)

  # This lets you use it with less RAM and faster processing.
  # But it will no longer be possible to update the detector with new training 
  # samples
  phraser = Phraser(phrases)

  # Merging multi-word expressions in the tokenization
  clean_corpus = []
  for sentence in tokenized_sentences:
    clean_corpus.append(phraser[sentence])
  
  return clean_corpus

In [29]:
cleaned_pol = clean_corpus(df_pol.text.array, threshold = 1000, tweets=True)
cleaned_eco = clean_corpus(df_eco.text.array,  threshold = 100, tweets=False)
cleaned_movie = clean_corpus(df_movies.overview.dropna().array,  threshold = 1000)

100%|██████████| 44512/44512 [00:38<00:00, 1146.55it/s]


In [30]:
print(cleaned_pol[184])
print(cleaned_eco[0])
print(cleaned_movie[42723])

['april', 'is', 'sexual_assault', 'awareness', 'month', '.']
['new_york', '--', 'yields', 'on', 'most', 'certificates', 'of', 'deposit', 'offered', 'by', 'major', 'banks', 'dropped', 'more', 'than', 'a', 'tenth', 'of', 'a', 'percentage_point', 'in', 'the', 'latest', 'week', ',', 'reflecting', 'the', 'overall', 'decline', 'in', 'short-term', 'interest_rates.', '<', '/br', '>', '<', '/br', '>', 'on', 'small-denomination', ',', 'or', '``', 'consumer', ',', "''", 'cds', 'sold', 'directly', 'by', 'banks', ',', 'the', 'average', 'yield', 'on', 'six-month', 'deposits', 'fell', 'to', '5.49', '%', 'from', '5.62', '%', 'in', 'the', 'week', 'ended', 'yesterday', ',', 'according', 'to', 'an', '18-bank', 'survey', 'by', 'banxquote', 'money', 'markets', ',', 'a', 'wilmington', ',', 'del.', ',', 'information', 'service.', '<', '/br', '>', '<', '/br', '>', 'on', 'three-month', '``', 'consumer', "''", 'deposits', ',', 'the', 'average', 'yield', 'sank', 'to', '5.29', '%', 'from', '5.42', '%', 'the', 'we

### Word2Vec

In [32]:
cpu = cpu_count()
print('The virtual instance has {} cpus, that will be used to train the word2vec model'.format(cpu))

# We will just get the "WordVectors" parameter from the trained Word2Vec model.
# Otherwise, we could continue training with some more exemples that could be
# fed on the fly to the model.
print("Training the political W2V ...")
pol = Word2Vec(cleaned_pol, size=100, window=5, min_count=3, workers=cpu)
pol.train(cleaned_pol, total_examples=len(cleaned_pol), epochs=10)
pol_wv = pol.wv
print("Training the economics W2V ...")
eco = Word2Vec(cleaned_eco, size=100, window=5, min_count=3, workers=cpu)
eco.train(cleaned_eco, total_examples=len(cleaned_eco), epochs=10)
eco_wv = eco.wv
print("Training the movies' W2V ...")
mo = Word2Vec(cleaned_movie, size=100, window=5, min_count=3, workers=cpu)
mo.train(cleaned_movie, total_examples=len(cleaned_movie), epochs=10)
mo_wv = mo.wv

The virtual instance has 8 cpus, that will be used to train the word2vec model
Training the political W2V ...
Training the economics W2V ...
Training the movies' W2V ...
